In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Utility to generate random BST

In [4]:
def generateBST(N: int, min_: int, max_: int) -> BinaryTreeNode|None:
    def insert(curr: BinaryTreeNode|None, n: int) -> BinaryTreeNode:
        if not curr:
            curr = BinaryTreeNode(n)
        elif curr.val < n:
            curr.right = insert(curr.right, n)
        else:
            curr.left = insert(curr.left, n)

        return curr

    assert N <= max_ - min_, "Number of available samples must be >= N"
    root: BinaryTreeNode|None = None
    for n in np.random.choice(np.arange(min_, max_), size=N, replace=False):
        root = insert(root, n)

    return root

LC Medium: Longest Substring with atleast K repeating characters

In [5]:
# https://leetcode.com/problems/longest-substring-with-at-least-k-repeating-characters/
# https://leetcode.com/problems/longest-substring-with-at-least-k-repeating-characters/submissions/1286506890
def longestSubstring(s: str, k: int) -> int:
    """
    Count the freq of all characters, split on the characters with freq < k
    """
    N = len(s)

    # Count the freq
    freq: collections.Counter = collections.Counter(s)
    is_valid = not any(map(lambda x: x < k, freq.values()))

    # If the entire string is okay, return length as answer
    if is_valid:
        return N

    # Split on the invalid characters and backtrack inside
    else:
        max_length = start_idx = 0
        for curr_idx in range(N):
            if 0 < freq[s[curr_idx]] < k:
                max_length = max(max_length, longestSubstring(s[start_idx:curr_idx], k))
                start_idx = curr_idx + 1

        max_length = max(max_length, longestSubstring(s[start_idx:], k))
        return max_length

# Tesitng the solution
assert longestSubstring("aaabb", 3) == 3
assert longestSubstring("ababbc", 2) == 5
assert longestSubstring("bbaaa", 3) == 3

LC Virtual - 4: Weekly Contest 5
https://leetcode.com/contest/leetcode-weekly-contest-5/

In [6]:
# Q4: Frog Jump
def canCrossRecursive(stones: list[int]) -> bool:
    @functools.cache
    def backtrack(n: int, jump: int) -> bool:
        if jump < 0:
            return False
        elif n == stones[-1]:
            return True
        else:
            if n + jump - 1 in positions and backtrack(n + jump - 1, jump - 1):
                return True
            elif jump > 0 and n + jump in positions and backtrack(n + jump, jump):
                return True
            elif n + jump + 1 in positions and backtrack(n + jump + 1, jump + 1):
                return True
            else:
                return False

    N = len(stones)
    positions: dict[int, int] = {n: i for i, n in enumerate(stones)}
    return len(stones) <= 1 or backtrack(stones[0], 0)

# Testing the solution
assert canCrossRecursive([0,1,3,5,6,8,12,17]) == True
assert canCrossRecursive([0,1,2,3,4,8,9,11]) == False
assert canCrossRecursive([]) == True
assert canCrossRecursive([100]) == True
assert canCrossRecursive([0,2]) == False

In [7]:
def canCrossTabulation(stones: list[int]) -> bool:
    N = len(stones)
    positions: dict[int, int] = {n: i for i, n in enumerate(stones)}
    dp: list[list[bool]] = [[False if stones[i] != stones[-1] else True for jump in range(N + 2)] for i in range(N)]
    for i in range(N - 2, -1, -1):
        for jump in range(N):
            n = stones[i]
            if n + jump - 1 in positions:
                dp[i][jump] = dp[i][jump] or dp[positions[n + jump - 1]][jump - 1]
            if jump > 0 and n + jump in positions:
                dp[i][jump] = dp[i][jump] or dp[positions[n + jump]][jump]
            if n + jump + 1 in positions:
                dp[i][jump] = dp[i][jump] or dp[positions[n + jump + 1]][jump + 1]

    return len(stones) <= 1 or dp[0][0]

# Testing the solution
assert canCrossTabulation([0,1,3,5,6,8,12,17]) == True
assert canCrossTabulation([0,1,2,3,4,8,9,11]) == False
assert canCrossTabulation([]) == True
assert canCrossTabulation([100]) == True
assert canCrossTabulation([0,2]) == False

In [8]:
# Q3: Remove K digits
def removeKdigits(num: str, k: int) -> str:
    pass

In [9]:
# Q1: Find Nth Digit
def findNthDigit(N: int) -> int:
    def countDigitsBefore(n: int) -> int:
        if n < 10:
            return n
        else:
            n_digits = math.floor(math.log10(n)) + 1
            smallest_n_digit_number = 10 ** (n_digits - 1)
            n_digits_before = (n - smallest_n_digit_number + 1) * n_digits
            return n_digits_before + countDigitsBefore(smallest_n_digit_number - 1)

    # Binary search
    low, high = 1, N
    while low <= high:
        mid = (low + high) // 2
        count = countDigitsBefore(mid)
        if count == N:
            return mid % 10
        elif count < N:
            low = mid + 1
        else:
            high = mid - 1

    delta = countDigitsBefore(low) - N
    return int(str(low)[-delta-1])

# Testing the solution
assert findNthDigit(11) == 0
assert findNthDigit(3) == 3

In [10]:
# Q2: Binary Watch
def readBinaryWatch(turnedOn: int) -> set[str]:
    watch: list[bool] = [False for i in range(10)]
    result: set[str] = set()
    def backtrack(lights: int) -> None:
        if lights == 0:
            hour = minute = 0
            for i in range(10):
                if watch[i]:
                    if i < 4:
                        hour += 2 ** (3 - i)
                    else:
                        minute += 2 ** (9 - i)

            if 0 <= hour < 12 and 0 <= minute <= 59:
                result.add(f"{hour}:{str(minute).zfill(2)}")

        else:
            for i in range(10):
                if not watch[i]:
                    watch[i] = True
                    backtrack(lights - 1)
                    watch[i] = False

    backtrack(turnedOn)
    return result

# Testing the solution
assert sorted(readBinaryWatch(1)) == sorted(["0:01","0:02","0:04","0:08","0:16","0:32","1:00","2:00","4:00","8:00"])

Minimum no of moves to seat everyone 
https://leetcode.com/problems/minimum-number-of-moves-to-seat-everyone

In [11]:
def minMovesToSeat(seats: list[int], students: list[int]) -> int:
    # Compute length
    N = len(seats)

    # Sort both to ensure that we get the minimal distances
    seats.sort()
    students.sort()

    # Compute abs distance between seats and students
    moves = 0
    for i in range(N):
        moves += abs(seats[i] - students[i])
    return moves

# Testing the solution
assert minMovesToSeat([3,1,5], [2,7,4]) == 4
assert minMovesToSeat([2,2,6,6], [1,3,2,6]) == 4

LC Medium: Minimum increment to make array unique
https://leetcode.com/problems/minimum-increment-to-make-array-unique

In [12]:
def minIncrementForUnique(nums: list[int]) -> int:
    # Count freq & unique values present
    counts: dict[int, int] = dict()
    unique: set[int] = set()
    for n in nums:
        counts[n] = counts.get(n, 0) + 1
        unique.add(n)

    # If count of any number is greater than 1, shift right
    moves = 0
    while unique:
        key = unique.pop()
        count = counts.pop(key)
        if count > 1:
            shift = count - 1
            moves += shift
            counts[key + 1] = counts.get(key + 1, 0) + shift
            unique.add(key + 1)

    return moves

# Testing the solution
assert minIncrementForUnique([1,2,2]) == 1
assert minIncrementForUnique([3,2,1,2,1,7]) == 6
assert minIncrementForUnique([1,2,2,5,5]) == 2

Maximum Gap
https://leetcode.com/problems/maximum-gap/

In [13]:
# https://leetcode.com/problems/maximum-gap/submissions/1288173457/
def maximumGap(nums: list[int]) -> int:
    """
    1. Maximum gap is always greater than or equal to ceil((max - min) / N)
        - Divide N nums into N - 1 groups
        - If evenly distributed, each elements would be in a different bucket (1 element in each bucket)
        - If unevenly distributed, two or more buckets would have multiple elements. Some buckets may not have any elements - max gap could occur betweeen left and right of the missing buckets
    2. Consequently we only need to store the min and max of each bucket
    """
    N, high, low = len(nums), max(nums), min(nums)
    bucket_size = math.ceil((high - low) / N)
    buckets: dict[int, tuple[int, int]] = {}

    # Edge case
    if bucket_size == 0:
        return 0

    # Group the numbers into buckets
    for n in nums:
        index = n // bucket_size
        min_, max_ = buckets.get(index, ((index + 1) * bucket_size, (index - 1) * bucket_size))
        min_, max_ = min(min_, n), max(max_, n)
        buckets[index] = (min_, max_)

    # Iterate through each and find the max gap
    curr, end = low // bucket_size, high // bucket_size
    max_gap = 0
    while curr < end:
        next_ = curr + 1
        while next_ not in buckets:
            next_ += 1
        curr, max_gap = next_, max(max_gap, buckets[next_][0] - buckets[curr][1])

    return max_gap

# Testing the solution
assert maximumGap([3,14,15,83,6,4,19,20,40]) == 43
assert maximumGap([3,6,9,1]) == 3
assert maximumGap([10]) == 0

LC Virtual - 5: Weekly Contest 6
https://leetcode.com/contest/leetcode-weekly-contest-6/

In [14]:
# Q1: Sum of left leaves
def sumOfLeftLeaves(root: BinaryTreeNode) -> int:
    def backtrack(curr: BinaryTreeNode) -> tuple[int, int]:
        if not curr:
            return 0, -1
        else:
            left_sum, left_type = backtrack(curr.left)
            right_sum, right_type = backtrack(curr.right)
            curr_type, total = 0 if left_type == -1 and right_type == -1 else 1, 0
            if curr_type == 0:
                total += curr.val
            if left_type != -1:
                total += left_sum
            if right_type >= 1:
                total += right_sum
            result = total, curr_type
            return result

    return backtrack(root)[0] if root.left or root.right else 0

# Testing the solution
assert sumOfLeftLeaves(BinaryTreeNode.from_array([3,9,20,None,None,15,7])) == 24
assert sumOfLeftLeaves(BinaryTreeNode.from_array([1])) == 0

In [15]:
# Q2: Convert to hexadecimal
def toHex(num: int) -> str:
    # If negative, find hex for its 2's compliment
    if num < 0:
        bit_length = 32
        num &= (2 << bit_length - 1) - 1

    # Use division method
    division_result: list[int] = []
    while num >= 16:
        num, rem = num // 16, num % 16
        division_result.append(rem)
    division_result.append(num)
    division_result.reverse()

    # Convert to hexadecimal string
    mapping = {i: str(i) for i in range(0, 10)} | {i: chr(ord('a') + i - 10) for i in range(10, 16)}
    return ''.join(map(lambda x: mapping[x], division_result))

# Testing the solution
assert toHex(26) == "1a"
assert toHex(-1) == "ffffffff"

In [16]:
# Q3: Queue Reconstruction by Height
def reconstructQueue(people: list[list[int]]) -> list[list[int]]:
    N = len(people)

    # Sort to find the relative height ordering
    people.sort()

    # Iterate from shortest to tallest, keeping track of number of person with height equal to curr
    eq_front = []
    prev, eq_count = -1, 0
    for i in range(N):
        if people[i][0] == prev:
            eq_count += 1
        else:
            eq_count = 0
        prev = people[i][0]
        eq_front.append(eq_count)

    # Insert into queue in correct order
    result: list[list[int]] = []
    while eq_front:
        actual, (height, expected) = eq_front.pop(), people.pop()
        if expected == actual:
            result.append([height, expected])
        else:
            temp: list[list[int]] = []
            for i in range(expected - actual):
                temp.append(result.pop())
            result.append([height, expected])
            while temp:
                result.append(temp.pop())

    result.reverse()
    return result

# Testing the solution
assert reconstructQueue([[7,0],[4,4],[7,1],[5,0],[6,1],[5,2]]) == [[5,0],[7,0],[5,2],[6,1],[4,4],[7,1]]
assert reconstructQueue([[6,0],[5,0],[4,0],[3,2],[2,2],[1,4]]) == [[4,0],[5,0],[2,2],[3,2],[1,4],[6,0]]

Trapping Rain water - II (Upsolving)
https://leetcode.com/problems/trapping-rain-water-ii/

In [17]:
# Q4: Trapping Rain water - II
# https://leetcode.com/problems/trapping-rain-water-ii/solutions/1138028/python3-visualization-bfs-solution-with-explanation
def trapRainWater(heightMap: list[list[int]]) -> int:
    N, M = len(heightMap), len(heightMap[0])
    visited: set[tuple[int, int]] = set()

    # Add boundary cells to heap, heap: Height, i, j
    heap: list[tuple[int, int, int]] = []
    for i in range(N):
        for j in range(M):
            if i in (0, N - 1) or j in (0, M - 1):
                heapq.heappush(heap, (heightMap[i][j], i, j))
                visited.add((i, j))

    level = volume = 0
    while heap:
        height, x, y = heapq.heappop(heap)
        level = max(level, height)
        for i, j in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
            if 0 <= i < N and 0 <= j < M and (i, j) not in visited:
                next_height = heightMap[i][j]
                if next_height < level:
                    volume += level - next_height
                visited.add((i, j))
                heapq.heappush(heap, (next_height, i, j))

    return volume

# Testing the solution
assert trapRainWater([[1,4,3,1,3,2],[3,2,1,3,2,4],[2,3,3,2,3,1]]) == 4
assert trapRainWater([[3,3,3,3,3],[3,2,2,2,3],[3,2,1,2,3],[3,2,2,2,3],[3,3,3,3,3]]) == 10
assert trapRainWater([[12,13,1,12],[13,4,13,12],[13,8,10,12],[12,13,12,12],[13,13,13,13]]) == 14
assert trapRainWater([[5,5,5,1],[5,1,1,5],[5,1,5,5],[5,2,5,8]]) == 3
assert trapRainWater([[14,17,18,16,14,16],[17,3,10,2,3,8],[11,10,4,7,1,7],[13,7,2,9,8,10],[13,1,3,4,8,6],[20,3,3,9,10,8]]) == 25

https://leetcode.com/contest/leetcode-weekly-contest-8/problems/pacific-atlantic-water-flow/

In [18]:
def pacificAtlantic(heights: list[list[int]]) -> set[tuple[int, int]]:
    def DFS(to_visit: list[tuple[int, int]]) -> set[tuple[int, int]]:
        visited: set[tuple[int, int]] = set()
        while to_visit:
            x, y = to_visit.pop()
            visited.add((x, y))
            for i, j in [(x, y + 1), (x, y - 1), (x + 1, y), (x - 1, y)]:
                if 0 <= i < N and 0 <= j < M and (i, j) not in visited and heights[i][j] >= heights[x][y]:
                    to_visit.append((i, j))

        return visited

    N, M = len(heights), len(heights[0])

    pacific: set[tuple[int, int]] = DFS([(0, i) for i in range(M)] + [(i, 0) for i in range(N)])
    atlantic: set[tuple[int, int]] = DFS([(N - 1, i) for i in range(M)] + [(i, M - 1) for i in range(N)])

    return set.intersection(pacific, atlantic)

# Testing the soluiton
pacificAtlantic([[1,2,2,3,5],[3,2,3,4,4],[2,4,5,3,1],[6,7,1,4,5],[5,1,1,2,4]])

{(0, 4), (1, 3), (1, 4), (2, 2), (3, 0), (3, 1), (4, 0)}

In [19]:
# https://leetcode.com/contest/leetcode-weekly-contest-8/problems/sentence-screen-fitting/
def sentenceScreenFitting(sentence: list[str], rows: int, cols: int) -> int:
    n_words = len(sentence)
    word_length: list[int] = list(map(len, sentence))
    word_idx = n_sentences = 0

    row_pending = rows
    while row_pending > 0:
        col_pending = cols
        while col_pending > 0:

            if word_length[word_idx] <= col_pending:
                col_pending -= word_length[word_idx]
                word_idx += 1

                if col_pending > 0:
                    col_pending -= 1

            else:
                col_pending = 0

            if word_idx == n_words:
                word_idx = 0
                n_sentences += 1

        row_pending -= 1

    return n_sentences

# Testing the solution
assert sentenceScreenFitting(["hello","world"], 2, 8) == 1
assert sentenceScreenFitting(["a", "bcd", "e"], 3, 6) == 2
assert sentenceScreenFitting(["i","had","apple","pie"], 4, 5) == 1

Codeforces: https://codeforces.com/problemset/problem/1843/C

In [20]:
def sumBinaryTree(n: int) -> int:
    if n == 1:
        return 1
    elif n % 2 == 0:
        return n + sumBinaryTree(n // 2)
    else:
        return n + sumBinaryTree((n - 1) // 2)

# Testing the solution
assert sumBinaryTree(37) == 71
assert sumBinaryTree(10000000000000000) == 19999999999999980
assert sumBinaryTree(15) == 26

In [21]:
def insertionInBST(curr: BinaryTreeNode, val: int):
    if not curr:
        curr = BinaryTreeNode(val)
    elif curr.val < val:
        curr.right = insertionInBST(curr.right, val)
    else:
        curr.left = insertionInBST(curr.left, val)
    return curr

# Testing the solution
assert insertionInBST(BinaryTreeNode.from_array([10, 2, 12, 1, 3, None, 13]), 11).to_list() == [10, 2, 12, 1, 3, 11, 13]

King Escape: https://codeforces.com/problemset/problem/1033/A

In [22]:
def kingEscape(N: int, queen: tuple[int, int], king: tuple[int, int], target: tuple[int, int]) -> bool:
    minX, maxX = min(king[0], target[0]), max(king[0], target[0])
    minY, maxY = min(king[1], target[1]), max(king[1], target[1])
    return not (minX < queen[0] < maxX or minY < queen[1] < maxY)

# Testing the solution
assert kingEscape(8, (4, 4), (1, 3), (3, 1)) == True
assert kingEscape(8, (4, 4), (2, 3), (1, 6)) == False
assert kingEscape(8, (3, 5), (1, 2), (6, 1)) == False

LC Medium: https://leetcode.com/problems/most-profit-assigning-work/submissions/1292541583

In [23]:
def maxProfitAssignment(difficulty: list[int], profit: list[int], worker: list[int]) -> int:
    # Sort workers by their capacity
    worker.sort()

    # Sort jobs available based on their difficuly
    jobs: list[tuple[int, int]] = sorted(zip(difficulty, profit))

    # Use a two pointer approach - iterate through jobs adding to our maxHeap until diff cross what curr worker can handle
    heap: list[int] = []
    maxProfit = i = 0
    for max_difficulty in worker:
        while i < len(jobs) and jobs[i][0] <= max_difficulty:
            heapq.heappush(heap, -jobs[i][1])
            i += 1
        maxProfit += -heap[0] if heap else 0

    return maxProfit

# Testing the solution
assert maxProfitAssignment([2,4,6,8,10], [10,20,30,40,50], [4,5,6,7]) == 100
assert maxProfitAssignment([85,47,57], [24,66,99], [40,25,25]) == 0
assert maxProfitAssignment([13,37,58], [4,90,96], [34,73,45]) == 190
assert maxProfitAssignment([68,35,52,47,86], [67,17,1,81,3], [92,10,85,84,82]) == 324

https://codeforces.com/problemset/problem/1829/E

In [24]:
class DisjointSetLakes:
    def __init__(self, N: int, M: int, grid: list[list[int]]):
        self.N, self.M = N, M
        self.grid = grid
        self.parent: dict[tuple[int, int], tuple[int, int]] = dict()
        self.size: dict[tuple[int, int], int] = dict()
        self.max_volume = 0
        self.iterate_grid()

    def is_valid(self, i: int, j: int) -> bool:
        return 0 <= i < self.N and 0 <= j < self.M

    def iterate_grid(self) -> None:
        for i in range(self.N):
            for j in range(self.M):
                if self.grid[i][j] > 0:
                    top, left = (i - 1, j), (i, j - 1)
                    if self.is_valid(*top) and self.grid[top[0]][top[1]] > 0:
                        self.unionBySize((i, j), top)
                    if self.is_valid(*left) and self.grid[left[0]][left[1]] > 0:
                        self.unionBySize((i, j), left)
                    self.max_volume = max(self.max_volume, self.size[self.findUltimateParent((i, j))])

    def unionBySize(self, c1: tuple[int, int], c2: tuple[int, int]) -> None:
        up1, up2 = self.findUltimateParent(c1), self.findUltimateParent(c2)
        s1, s2 = self.size[up1], self.size[up2]
        if up1 == up2:
            return
        elif s1 > s2:
            self.parent[up2] = up1
            self.size[up1] += self.size[up2]
        else:
            self.parent[up1] = up2
            self.size[up2] += self.size[up1]

        # Update the max volume
        self.max_volume = max(self.max_volume, self.size[up1], self.size[up2])

    def findUltimateParent(self, cell: tuple[int, int]) -> tuple[int, int]:
        if cell not in self.parent:
            self.parent[cell] = cell
            self.size[cell] = self.grid[cell[0]][cell[1]]

        if cell == self.parent[cell]:
            return cell
        else:
            self.parent[cell] = self.findUltimateParent(self.parent[cell])
            return self.parent[cell]

# Testing the solution
assert DisjointSetLakes(3, 3, [[1,2,0],[3,4,0],[0,0,5]]).max_volume == 10
assert DisjointSetLakes(1, 1, [[9]]).max_volume == 9
assert DisjointSetLakes(3, 3, [[0,1,1],[1,0,1],[1,1,1]]).max_volume == 7
assert DisjointSetLakes(5, 5, [[1,1,1,1,1],[1,0,0,0,1],[1,0,5,0,1],[1,0,0,0,1],[1,1,1,1,1]]).max_volume == 16

In [25]:
def maximumVolumeLake(N: int, M: int, grid: list[list[int]]) -> int:
    def is_valid(i: int, j: int) -> bool:
        return 0 <= i < N and 0 <= j < M

    def DFS(i: int, j: int) -> int:
        stack: list[tuple[int, int]] = [(i, j)]
        volume: int = 0
        while stack:
            x, y = stack.pop()
            volume += grid[x][y]
            grid[x][y] = 0
            for i, j in [(x, y - 1), (x, y + 1), (x - 1, y), (x + 1, y)]:
                if is_valid(i, j) and grid[i][j] > 0:
                    stack.append((i, j))

        return volume

    max_volume = 0
    for i in range(N):
        for j in range(M):
            if grid[i][j] > 0:
                max_volume = max(max_volume, DFS(i, j))

    return max_volume

# Testing the solution
assert maximumVolumeLake(3, 3, [[1,2,0],[3,4,0],[0,0,5]])
assert maximumVolumeLake(1, 1, [[9]]) == 9
assert maximumVolumeLake(3, 3, [[0,1,1],[1,0,1],[1,1,1]]) == 7
assert maximumVolumeLake(5, 5, [[1,1,1,1,1],[1,0,0,0,1],[1,0,5,0,1],[1,0,0,0,1],[1,1,1,1,1]]) == 16

LC Virtual: https://leetcode.com/contest/leetcode-weekly-contest-10/

In [26]:
# Q1: Island Perimeter
def islandPerimeter(grid: list[list[int]]) -> int:
    """
    Iterate through grid, count the number of sides covered with water
    """
    N, M = len(grid), len(grid[0])
    perimeter = 0
    for i in range(N):
        for j in range(M):
            if grid[i][j] == 1:
                for x, y in ((i, j - 1), (i, j + 1), (i - 1, j), (i + 1, j)):
                    if x < 0 or y < 0 or x >= N or y >= M or grid[x][y] == 0:
                        perimeter += 1

    return perimeter

# Testing the solution
assert islandPerimeter([[0,1,0,0],[1,1,1,0],[0,1,0,0],[1,1,0,0]]) == 16
assert islandPerimeter([[1]]) == 4
assert islandPerimeter([[1,0]]) == 4

In [27]:
# Q2: Can I Win?
def canIWinBrute(maxChoosableInteger: int, desiredTotal: int) -> bool:
    @functools.cache
    def backtrack(picked: tuple[bool, ...], total: int) -> bool:
        for pick in range(1, maxChoosableInteger + 1):
            if not picked[pick]:
                if pick >= total:
                    return True
                else:
                    new_picked = list(picked)
                    new_picked[pick] = True
                    otherWin = backtrack(tuple(new_picked), total - pick)
                    if not otherWin:
                        return True

        return False

    if (maxChoosableInteger * (maxChoosableInteger + 1)) // 2 < desiredTotal:
        return False
    else:
        result = backtrack(tuple(False for i in range(maxChoosableInteger + 1)), desiredTotal)
        return result

# Testing the solution
assert canIWinBrute(10, 11) == False
assert canIWinBrute(10, 0) == True
assert canIWinBrute(10, 40) == False

In [28]:
def canIWin(maxChoosableInteger: int, desiredTotal: int) -> bool:
    @functools.cache
    def backtrack(picked: int, total: int) -> bool:
        for pick in range(1, maxChoosableInteger + 1):
            mask = 1 << pick
            if not (picked & mask):
                if pick >= total:
                    return True
                else:
                    otherWin = backtrack(picked | mask, total - pick)
                    if not otherWin:
                        return True

        return False

    if (maxChoosableInteger * (maxChoosableInteger + 1)) // 2 < desiredTotal:
        return False
    else:
        result = backtrack(0, desiredTotal)
        return result

# Testing the solution
assert canIWin(10, 11) == False
assert canIWin(10, 0) == True
assert canIWin(10, 40) == False

In [29]:
# Q3: Min Moves 2
def minMoves2(nums: list[int]) -> int:
    # Find the middle element, convert all elements to this
    nums.sort()

    # Figure out the element in the middle
    N = len(nums)
    mid = nums[N // 2]

    # Count total moves
    moves = 0
    for n in nums:
        moves += abs(mid - n)

    return moves

# Testing the solution
assert minMoves2([1,2,3]) == 2
assert minMoves2([1,10,2,9]) == 16
assert minMoves2([1]) == 0

Find the duplicate number

In [30]:
# This modifies the array
def findDuplicateBetter(nums: list[int]) -> int:
    idx = 0
    while nums[idx] != -1:
        nums[idx], idx = -1, nums[idx]

    return idx

# Testing the solution
assert findDuplicateBetter([1,3,4,2,2]) == 2
assert findDuplicateBetter([3,1,3,4,2]) == 3
assert findDuplicateBetter([3,3,3,3,3]) == 3

In [31]:
def findDuplicate(nums: list[int]) -> int:
    slow, fast = nums[0], nums[nums[0]]

    # Find point of collision
    while fast != slow:
        slow, fast = nums[slow], nums[nums[fast]]

    # Find the start point of the loop
    slow = 0
    while slow != fast:
        slow, fast = nums[slow], nums[fast]

    return slow

# Testing the solution
assert findDuplicate([1,3,4,2,2]) == 2
assert findDuplicate([3,1,3,4,2]) == 3
assert findDuplicate([3,3,3,3,3]) == 3

Min cost tree from leaf nodes: https://leetcode.com/problems/minimum-cost-tree-from-leaf-values/solutions/340014/greedy-python-solution/

In [32]:
def mctFromLeafValues(arr: list[int]) -> int:
    """
    1. Take the smallest node from inorder arr.
    2. Combine it with the smallest of its two neighbours. Pop the smallest, add smallest * smaller neighbour to total.
    We pop it out since starting with next step, the min value would not be considered, since larger value would always be the 'largest' of the subtree.
    3. Repeat until we have one element left, each time we compute the min index and combine it with its larger neighbour.
    """
    def findMinNode(nums: list[int]) -> float:
        idx, val = 0, nums[0]
        for i in range(len(nums)):
            if nums[i] < val:
                idx, val = i, nums[i]

        parent_node_val: float = val * min(nums[idx - 1] if idx - 1 >= 0 else math.inf, nums[idx + 1] if idx + 1 < len(nums) else math.inf)
        nums.pop(idx)
        return parent_node_val

    non_leaf_sum: float = 0.
    while len(arr) > 1:
        non_leaf_sum += findMinNode(arr)

    return int(non_leaf_sum)

# Testing the solution
assert mctFromLeafValues([6,2,4]) == 32
assert mctFromLeafValues([4,11]) == 44

Time Based Key-Value Store

In [33]:
import bisect

class TimeMap:
    def __init__(self) -> None:
        self.map: dict[str, list[tuple[int, str]]] = dict()

    def set(self, key: str, value: str, timestamp: int) -> None:
        values: list[tuple[int, str]] = self.map.get(key, [])
        idx = bisect.bisect(values, (timestamp, ""))
        if 0 <= idx < len(values) and values[idx][0] == timestamp:
            values[idx] = (timestamp, value)
        else:
            bisect.insort(values, (timestamp, value))
        self.map[key] = values

    def get(self, key: str, timestamp: int) -> str:
        values: list[tuple[int, str]] = self.map.get(key, [])
        idx = bisect.bisect(values, (timestamp, ""))
        if 0 <= idx < len(values) and values[idx][0] == timestamp:
            return values[idx][1]
        elif idx - 1 >= 0:
            return values[idx - 1][1]
        else:
            return ""

# Testing the solution
obj: TimeMap = TimeMap()
result: list[str] = []
for params in [["love","high",10],["love","low",20],["love",5],["love",10],["love",15],["love",20],["love",25]]:
    if len(params) == 3:
        obj.set(str(params[0]), str(params[1]), int(params[2])) # type: ignore
        result.append("")
    else:
        result.append(obj.get(str(params[0]), int(params[1]))) # type: ignore

print(result)

['', '', '', 'high', 'high', 'low', 'low']


https://codeforces.com/problemset/problem/1979/A

In [34]:
def guessMaximum(nums: list[int]) -> int:
    # Slide through nums (window size of 2), find the minimum maximum
    minimum: float = math.inf
    for i in range(len(nums) - 1):
        minimum = min(minimum, max(nums[i], nums[i + 1]))

    return int(minimum) - 1

# Testing the solution
assert guessMaximum([2,4,1,7]) == 3
assert guessMaximum([1,1]) == 0
assert guessMaximum([37,8,16]) == 15

Top K Frequent elements

In [35]:
def topKFrequentBrute(nums: list[int], k: int) -> list[int]:
    freq: collections.Counter = collections.Counter(nums)
    heap: list[tuple[int, int]] = []
    for n, f in freq.items():
        if len(heap) == k:
            heapq.heappushpop(heap, (f, n))
        else:
            heapq.heappush(heap, (f, n))

    result: list[int] = []
    while heap:
        f, n = heapq.heappop(heap)
        result.append(n)

    result.reverse()
    return result

# Testing the solution
assert sorted(topKFrequentBrute([1,1,1,2,2,3,3,3,4,4,5], 2)) == [1, 3]

LC Virtual: https://leetcode.com/contest/leetcode-weekly-contest-11/

In [36]:
# Number of segments in a string
def countSegments(s: str) -> int:
    segments, space = 0, True
    for ch in s:
        if ch.isspace():
            segments += not space
            space = True
        else:
            space = False

    return segments + (not space)

# Testing the solution
assert countSegments("Python is a pretty cool language") == 6
assert countSegments(" Python is pretty    cool    ") == 4
assert countSegments("Hello") == 1

Q3: Unique substrings in a wrap around string
https://youtu.be/2nCTyDrRm4I?si=TP9X0ZwTJNhKHkl4

In [37]:
def findSubstringInWraproundString(s: str) -> int:
    """
    Initial thought process: Check the longest substring that can end at an index.
    This however doesn't account for duplicates.

    To handle duplicates we store the max length of substrings ending for english alphabet
    For eg: zabxabc
    z - 1
    a - 1
    b - 3
    x - 1
    a - 1
    b - 3
    c - 3

    Once completed, count += (arr[i] * arr[i + 1]) / 2
    """
    longest: dict[str, int] = dict()
    count = 0
    for i in range(len(s)):
        if i == 0 or ord(s[i]) - ord(s[i - 1]) == 1 or (s[i] == 'a' and s[i - 1] == 'z'):
            count += 1
        else:
            count = 1
        longest[s[i]] = max(longest.get(s[i], 0), count)

    # Count substrings
    return sum(longest.values())

# Testing the solution
assert findSubstringInWraproundString("zabxabc") == 10
assert findSubstringInWraproundString("cac") == 2
assert findSubstringInWraproundString("zab") == 6

Compress the string
https://www.naukri.com/code360/problems/compress-the-string_893402

In [38]:
def compressTheString(s: str):
    stack: list[tuple[str, int]] = []
    for ch in s:
        if not stack or stack[-1][0] != ch:
            stack.append((ch, 1))
        else:
            stack[-1] = (ch, stack[-1][1] + 1)

    result: list[str] = []
    for ch, freq in stack:
        result.append(f'{ch}{freq}' if freq > 1 else ch)
    return ''.join(result)

# Testing the solution
assert compressTheString("aaabbddccc") == "a3b2d2c3"
assert compressTheString("ggttttffffrreee") == "g2t4f4r2e3"

CN Weekly contest: 20th June 2024

In [39]:
def maximumElements(N: int, arr: list[int]) -> int:
    odd = even = 0
    for n in arr:
        if n % 2 == 1:
            odd += 1
        else:
            even += 1

    if odd == 0:
        return -1
    else:
        return even + odd - int(odd % 2 == 0)

# Testing the solution
assert maximumElements(4, [9,5,1,6]) == 4

In [40]:
# Supercomputer
def canAccess(N: int, K: int, arr: list[int]) -> int:
    arr.sort()
    count = 0
    for i in range(N - 2):
        for j in range(i + 1, N - 1):
            target = K ^ arr[i] ^ arr[j]
            left = bisect.bisect_left(arr, target, j + 1)
            right = bisect.bisect_right(arr, target, j + 1)
            if left < N and arr[left] == target:
                count += right - left

    return count

# Testing the solution
assert canAccess(6, 13, [2,2,8,7,7,9]) == 4
assert canAccess(7, 5, [7,2,5,5,2,4,7]) == 4
assert canAccess(6, 1, [5,6,5,2,1,7]) == 3

Maximum magnetic force between two balls

In [41]:
def maxDistance(positions: list[int], m: int) -> int:
    def is_possible(max_sum: int) -> bool:
        prev, k = -max_sum, m
        for n in positions:
            if n - prev >= max_sum:
                k, prev = k - 1, n

        return k <= 0

    # Sort the positions and use a binary search approach, greedily checking if maxsum is possible each time
    positions.sort()

    low, high = 1, positions[-1] - positions[0]
    while low <= high:
        mid = (low + high) // 2
        if is_possible(mid):
            low = mid + 1
        else:
            high = mid - 1

    return high

# Testing the solution
assert maxDistance([1,2,3,4,7], 3) == 3
assert maxDistance([5,4,3,2,1,1000000000], 2) == 999999999
assert maxDistance([5,4,3,2,1,1000000000], 3) == 4

LC Virtual: 21st June 2024
Link: https://leetcode.com/contest/smarking-algorithm-contest/

In [42]:
# Q1: Path Sum - III
def pathSum(root: BinaryTreeNode, targetSum: int) -> int:
    count = 0
    def backtrack(curr: BinaryTreeNode) -> dict[int, int]:
        nonlocal count
        if not curr:
            return dict()
        else:
            left: dict[int, int] = backtrack(curr.left)
            right: dict[int, int] = backtrack(curr.right)
            result: dict[int, int] = dict()
            for k, v in left.items():
                result[k + curr.val] = result.get(k + curr.val, 0) + v
            for k, v in right.items():
                result[k + curr.val] = result.get(k + curr.val, 0) + v
            result[curr.val] = result.get(curr.val, 0) + 1
            count += result.get(targetSum, 0)
            return result

    backtrack(root)
    return count

# Testing the solution
assert pathSum(BinaryTreeNode.from_array([10,5,-3,3,2,None,11,3,-2,None,1]), 8) == 3
assert pathSum(BinaryTreeNode.from_array([5,4,8,11,None,13,4,7,2,None,None,5,1]), 22) == 3

In [43]:
# Q2: Find all anagrams in a string
def findAnagrams(s: str, p: str) -> list[int]:
    def compare(freq1: dict[str, int], freq2: dict[str, int]) -> bool:
        for k, v in freq1.items():
            if k not in freq2 or v != freq2[k]:
                return False
        return True if freq1 else False

    NS, NP = len(s), len(p)
    p_freq: collections.Counter = collections.Counter(p)

    result: list[int] = []
    # J is not included in slice
    i = j = 0
    s_freq: dict[str, int] = {}
    while j <= NS:
        while j < NS and j - i < NP:
            s_freq[s[j]] = s_freq.get(s[j], 0) + 1
            j += 1

        comparison = compare(s_freq, p_freq)

        # Anagram
        if comparison == 1:
            result.append(i)
        if j < NS:
            s_freq[s[i]] = s_freq[s[i]] - 1
            s_freq[s[j]] = s_freq.get(s[j], 0) + 1
            if s_freq[s[i]] == 0:
                s_freq.pop(s[i])
        i, j = i + 1, j + 1

    return result

# Testing the solution
assert findAnagrams("cbaebabacd", "abc") == [0, 6]
assert findAnagrams("abab", "ab") == [0,1,2]
assert findAnagrams("aa", "bb") == []
assert findAnagrams("baa", "aa") == [1]

Upsolving: Kth Smallest in lexicographical ordering

In [44]:
# https://leetcode.com/problems/k-th-smallest-in-lexicographical-order/solutions/92242/concise-easy-to-understand-java-5ms-solution-with-explaination/
def findKthNumber(n: int, k: int) -> int:
    def countSteps(n1: int, n2: int) -> int:
        steps = 0
        while n1 <= n:
            steps += min(n2, n + 1) - n1
            n1, n2 = n1 * 10, n2 * 10

        return steps

    # Move k - 1 steps
    curr = 1
    while k > 1:
        steps = countSteps(curr, curr + 1)
        if steps <= k:
            curr, k = curr + 1, k - steps
        else:
            curr, k = curr * 10, k - 1

    return curr

# Testing the solution
assert findKthNumber(13, 2) == 10
assert findKthNumber(100, 5) == 12

GFG Hiring Challenge: 21st June 2024

In [45]:
def modifyMEX(N : int, arr : list[int]) -> int:
    freq: collections.Counter = collections.Counter(arr)
    prev = -1
    for i in range(0, int(1e6) + 1):
        if i not in freq:
            return prev
        else:
            prev = freq[i] if prev == -1 else min(freq[i], prev)

    return prev

# Testing the solution
assert modifyMEX(6, [0,1,2,2,3,5]) == 1
assert modifyMEX(4, [1,2,3,4]) == -1
assert modifyMEX(4, [0,1,1,1]) == 1

In [46]:
def countPairs(N: int, K: int, arr: list[int]) -> int:
    count: int = 0
    indices: dict[int, dict[int, int]] = dict()
    for i in range(N):
        idx, num = i + 1, arr[i]
        index_dp: dict[int, int] = indices.get(num, dict())
        idx %= K
        count += index_dp.get(K - idx if idx > 0 else 0, 0)
        index_dp[idx] = index_dp.get(idx, 0) + 1
        indices[num] = index_dp

    return int(count % (1e9 + 7))

# Testing the solution
assert countPairs(5, 3, [1,2,3,2,1]) == 2
assert countPairs(6, 4, [1,1,1,1,1,1]) == 3
assert countPairs(8, 2, [0,1,2,1,3,1,4,1]) == 6

In [47]:
def maxEdgeSumBrute(N: int, edges: list[list[int]]) -> int:
    """
    Naive solution: Apply modified djikistra's algorithm for each vertex.
    Increase sum for each pair.
    """
    def BFS(src: int) -> int:
        distances: dict[int, int] = dict()
        queue: collections.deque = collections.deque([(src, 0)])

        # Apply Dijkstra
        while queue:
            curr, dist = queue.popleft()
            distances[curr] = dist
            for next_, next_dist in adj[curr]:
                if next_ not in distances:
                    queue.append((next_, max(next_dist, dist)))

        edge_sum = 0
        for k, v in distances.items():
            edge_sum += v if k > src else 0

        return edge_sum

    # Convert edges into ADJ List
    adj: dict[int, list[tuple[int, int]]] = dict()
    for n1, n2, w in edges:
        n1_neighbours, n2_neighbours = adj.get(n1, []), adj.get(n2, [])
        n1_neighbours.append((n2, w))
        n2_neighbours.append((n1, w))
        adj[n1], adj[n2] = n1_neighbours, n2_neighbours

    sum_: int = 0
    for src in range(1, N + 1):
        sum_ += BFS(src)

    return sum_

# Testing the solution
assert maxEdgeSumBrute(3, [[1,2,2], [1,3,4]]) == 10
assert maxEdgeSumBrute(4, [[1,2,1], [2,4,3], [1,3,3]]) == 16

LC Daily question: Count number of nice subarrays

In [ ]:
def numberOfSubarrays(nums: list[int], K: int) -> int:
    def countSubArrays(k: int) -> int:
        subArrayCount = oddCount = 0
        i = j = 0
        while j < N:
            oddCount += nums[j] % 2
            # Shrink until valid
            while oddCount > k:
                oddCount -= nums[i] % 2
                i += 1
            subArrayCount += j - i + 1
            j += 1

        return subArrayCount

    N = len(nums)
    return countSubArrays(K) - countSubArrays(K - 1)

# Testing the solution
assert numberOfSubarrays([1,1,2,1,1], 3) == 2

Codeforces: Magnitude (https://codeforces.com/problemset/problem/1984/C1)

In [ ]:
def magnitudeBrute(arr: list[int]) -> int:
    @functools.cache
    def backtrack(i: int, c: int) -> int:
        if i >= N:
            return c
        else:
            opt1 = backtrack(i + 1, c + arr[i])
            opt2 = backtrack(i + 1, abs(c + arr[i]))
            return max(opt1, opt2)

    N = len(arr)
    return backtrack(0, 0)

# Testing the solution
assert magnitudeBrute([10, -9, -3, 4]) == 6
assert magnitudeBrute([1,4,3,4,1,4,3,4]) == 24

LC Biweekly: 21st June 2024

In [ ]:
# Q1
# find-minimum-operations-to-make-all-elements-divisible-by-three/description/
def minimumOperations(nums: list[int]) -> int:
    count = 0
    for n in nums:
        count += int(n % 3 in (1, 2))

    return count

# Testing the solution
assert minimumOperations([1,2,3,4]) == 3
assert minimumOperations([3,6,9]) == 0

In [ ]:
def minOperationsI(nums: list[int]) -> int:
    def flip(idx: int) -> None:
        for j in range(3):
            nums[idx + j] = 1 if nums[idx + j] == 0 else 0

    N, count = len(nums), 0
    for i in range(N):
        if nums[i] == 0:
            if i + 3 <= N:
                count += 1
                flip(i)
            else:
                return -1

    return count

# Testing the solution
assert minOperationsI([0,1,1,1,0,0]) == 3
assert minOperationsI([0,1,1,1]) == -1

In [ ]:
def minOperationsII(nums: list[int]) -> int:
    N, count, flip = len(nums), 0, False
    for i in range(N):
        nums[i] = nums[i] if not flip else int(not nums[i])
        if nums[i] == 0:
            flip = not flip
            count += 1

    return count

# Testing the solution
assert minOperationsII([0,1,1,0,1]) == 4
assert minOperationsII([1,0,0,0]) == 1

LC Hard: Sliding window Maximum

In [ ]:
def maxSlidingWindow(nums: list[int], k: int) -> list[int]:
    """
    Maintain a deque of indices in monotonically decreasing order.
    - Remove out of bound indices
    - Maintain monotonicity
    """

    N = len(nums)
    deq: collections.deque = collections.deque([])
    result: list[int] = []
    for i in range(N):

        # Maintain monotonicity
        while deq and nums[deq[-1]] < nums[i]:
            deq.pop()

        # Remove out of bound indices
        if deq and deq[0] < i - k + 1:
            deq.popleft()

        # Add current element index
        deq.append(i)

        # Add the current minimum to the result list
        if i >= k - 1:
            result.append(nums[deq[0]])

    return result

# Testing the solution
assert maxSlidingWindow([1,3,-1,-3,5,3,6,7], 3) == [3,3,5,5,6,7]

LC Weekly: 23rd June 2024

In [ ]:
# Q1
def minimumAverage(nums: list[int]) -> float:
    # Maintain two heaps
    min_heap: list[int] = list(nums)
    max_heap: list[int] = list(map(lambda x: -x, nums))

    # Heapify both heaps
    heapq.heapify(min_heap)
    heapq.heapify(max_heap)

    avg: float = math.inf
    while min_heap:
        min_, max_ = heapq.heappop(min_heap), -heapq.heappop(max_heap)
        avg = min(avg, (max_ + min_) / 2)

    return avg

# Testing the solution
assert minimumAverage([7,8,3,4,15,13,4,1]) == 5.5
assert minimumAverage([1,9,8,3,10,5]) == 5.5

In [ ]:
# Q2
def minimumArea(grid: list[list[int]]) -> int:
    N, M = len(grid), len(grid[0])
    min_x, min_y, max_x, max_y = N - 1, M - 1, 0, 0
    for i in range(N):
        for j in range(M):
            if grid[i][j] == 1:
                min_x, min_y, max_x, max_y = min(min_x, i), min(min_y, j), max(max_x, i), max(max_y, j)

    return (max_x - min_x + 1) * (max_y - min_y + 1)

# Testing the solution
assert minimumArea([[0,1,0],[1,0,1]]) == 6
assert minimumArea([[0,0,0],[1,0,1]]) == 3
assert minimumArea([[0,0],[1,0]]) == 1

In [ ]:
# Q3
def maximumTotalCostBrute(nums: list[int]) -> int:
    @functools.cache
    def backtrack(i: int) -> int:
        if i == N:
            return 0
        else:
            max_partition = sum(nums[i:])
            partition_sum, sign = 0, 1
            for j in range(i, N):
                partition_sum += nums[j] * sign
                max_partition = max(max_partition, partition_sum + backtrack(j + 1))
                sign = -1 if sign == 1 else 1

            return max_partition

    N = len(nums)
    return backtrack(0)

# Testing the solution
assert maximumTotalCostBrute([1,-2,3,4]) == 10
assert maximumTotalCostBrute([1,-1,1,-1]) == 4
assert maximumTotalCostBrute([0]) == 0
assert maximumTotalCostBrute([1,-1]) == 2
assert maximumTotalCostBrute([-15,8,12,-12]) == 17

In [ ]:
# Upsolving Q3
def maximumTotalCostRecursive(nums: list[int]) -> int:
    @functools.cache
    def backtrack(i: int, prev_flipped: bool) -> int:
        if i == N:
            return 0
        elif nums[i] < 0 and not prev_flipped:
            return max(-nums[i] + backtrack(i + 1, True), nums[i] + backtrack(i + 1, False))
        else:
            return nums[i] + backtrack(i + 1, False)

    N = len(nums)
    return backtrack(0, True)

# Testing the solution
assert maximumTotalCostRecursive([1,-2,3,4]) == 10
assert maximumTotalCostRecursive([1,-1,1,-1]) == 4
assert maximumTotalCostRecursive([1,-1]) == 2
assert maximumTotalCostRecursive([-15,8,12,-12]) == 17

In [ ]:
def maximumTotalCostTabulation(nums: list[int]) -> int:
    N = len(nums)
    dp: list[list[int]] = [[0, 0] for i in range(N + 1)]
    for i in range(N - 1, -1, -1):
        for j in range(2):
            if nums[i] < 0 and j == 0:
                dp[i][j] = max(-nums[i] + dp[i + 1][1], nums[i] + dp[i + 1][0])
            else:
                dp[i][j] = nums[i] + dp[i + 1][0]

    return dp[0][1]

# Testing the solution
assert maximumTotalCostTabulation([1,-2,3,4]) == 10
assert maximumTotalCostTabulation([1,-1,1,-1]) == 4
assert maximumTotalCostTabulation([1,-1]) == 2
assert maximumTotalCostTabulation([-15,8,12,-12]) == 17

In [ ]:
def maximumTotalCost(nums: list[int]) -> int:
    N = len(nums)
    dp: list[int] = [0, 0]
    for i in range(N - 1, -1, -1):
        dp = [max(-nums[i] + dp[1], nums[i] + dp[0]), nums[i] + dp[0]]

    return dp[1]

# Testing the solution
assert maximumTotalCost([1,-2,3,4]) == 10
assert maximumTotalCost([1,-1,1,-1]) == 4
assert maximumTotalCost([1,-1]) == 2
assert maximumTotalCost([-15,8,12,-12]) == 17